In [1]:
import sys
sys.path.insert(0, '../src/')

import pandas as pd
import os
import auto_co2 as co2
import importlib
importlib.reload(co2)

df = pd.read_csv('../data/')
df = co2.data.download_and_load_co2_data()
df = co2.data.dataviz_preprocess(df)

df['ElectricRange'] = df['ElectricRange'].fillna(0)
df['InnovativeEmissionReductionsWltp'] = df['InnovativeEmissionReductionsWltp'].fillna(0)


df = co2.data.soft_conditional_column_update(df, 'CategoryOf', 'm1', 'CategoryOf', 'M1')

2024-01-29 13:55:04.562820: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-29 13:55:04.579404: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


File ../data/raw/auto_co2_eur_21_raw.csv found, loading...
Loading data into a Pandas DataFrame...
Selecting countries...
ID                            0
Country                       0
VFN                     1004476
Mp                       317348
Mh                            0
Man                           0
MMS                     9920108
Tan                       24035
T                          3578
Va                        18211
Ve                        46725
Mk                          289
Cn                        13093
Ct                        20582
Cr                            0
r                             0
m (kg)                       32
Mt                      1936857
Enedc (g/km)            7502474
Ewltp (g/km)              83959
W (mm)                    17184
At1 (mm)                1561040
At2 (mm)                1565412
Ft                            0
Fm                         1795
ec (cm3)                1000610
ep (KW)                   89080
z (Wh/km)     

In [2]:
gb = df.groupby([
    'CommercialName', 
    'FuelType', 
    'CategoryOf', 
    'InnovativeTechnology', 
    'Co2EmissionsWltp', 
    'EnginePower', 
    'ElectricRange',
    'InnovativeEmissionReductionsWltp', 
    'MassRunningOrder', 
    'BaseWheel',
    'Pool'
]).size().reset_index().rename(columns={0: 'Count'})

gb['Share'] = gb['Count'] / gb['Count'].sum()

co2.styles.displayer(gb, n=10)
co2.styles.display_info(gb)
co2.styles.display_describe(df)

gb.sort_values(by=['Count'], ascending=False)


Save: True
Saved None styled DataFrame to output/tables


,CommercialName,FuelType,CategoryOf,InnovativeTechnology,Co2EmissionsWltp,EnginePower,ElectricRange,InnovativeEmissionReductionsWltp,MassRunningOrder,BaseWheel,Pool,Count,Share
0,,PETROL,M1,e5 29 37,124.000,73.000,0.000,0.000,1195.000,2580.000,HYUNDAI,1,0.000
1,,PETROL,M1,e5 29 37,125.000,73.000,0.000,0.000,1170.000,2580.000,HYUNDAI,1,0.000
2,,PETROL,M1,e5 29 37,127.000,61.000,0.000,0.000,1120.000,2580.000,HYUNDAI,1,0.000
3,116 / / D,DIESEL,M1,e24 29,119.000,85.000,0.000,0.000,1450.000,2670.000,BMW,1,0.000
4,116 / / D,DIESEL,M1,e24 29 37,119.000,85.000,0.000,0.000,1450.000,2670.000,BMW,1,0.000
5,116 / / D,DIESEL,M1,e24 3 28 29,119.000,85.000,0.000,0.000,1450.000,2670.000,BMW,1,0.000
6,116 / / D,DIESEL,M1,e24 3 28 29,121.000,85.000,0.000,0.000,1450.000,2670.000,BMW,1,0.000
7,116 / / D AUT,DIESEL,M1,e24 28 29,120.000,85.000,0.000,0.000,1460.000,2670.000,BMW,1,0.000
8,116 / / D AUT,DIESEL,M1,e24 28 29,121.000,85.000,0.000,0.000,1460.000,2670.000,BMW,3,0.000
9,116 / / D AUT,DIESEL,M1,e24 29 37,120.000,85.000,0.000,0.000,1460.000,2670.000,BMW,1,0.000


<class 'pandas.io.formats.style.Styler'>
Save: True
Saved None styled DataFrame to output/tables


,Column,Non-Null Count,Dtype
0,CommercialName,100365 non-null,object
1,FuelType,100365 non-null,object
2,CategoryOf,100365 non-null,object
3,InnovativeTechnology,100365 non-null,object
4,Co2EmissionsWltp,100365 non-null,float64
5,EnginePower,100365 non-null,float64
6,ElectricRange,100365 non-null,float64
7,InnovativeEmissionReductionsWltp,100365 non-null,float64
8,MassRunningOrder,100365 non-null,float64
9,BaseWheel,100365 non-null,float64


<class 'pandas.io.formats.style.Styler'>
Save: True
Saved None styled DataFrame to output/tables


,Info
0,"RangeIndex: 100365 entries, 0 to 100364"
1,Data columns (total 13 columns):
2,12 Share 100365 non-null float64
3,"dtypes: float64(7), int64(1), object(5)"
4,memory usage: 10.0+ MB


<class 'pandas.io.formats.style.Styler'>
Save: True
Saved None styled DataFrame to output/tables


,Colonne,count,mean,std,min,25%,50%,75%,max
0,MassRunningOrder,9446516.000,1473.574,328.471,860.000,1238.000,1414.000,1639.000,3950.000
1,Co2EmissionsWltp,9446516.000,115.291,52.424,0.000,112.000,126.000,141.000,572.000
2,BaseWheel,9446516.000,2651.791,165.024,984.000,2560.000,2640.000,2712.000,5840.000
3,EnginePower,9446516.000,100.711,48.412,9.000,70.000,92.000,110.000,1177.000
4,ElectricRange,9446516.000,29.541,99.797,0.000,0.000,0.000,0.000,808.000
5,FuelConsumption,9446516.000,3.313,3.003,0.000,0.000,4.500,5.600,99.000
6,AxleWidthSteering,7984515.000,1549.569,59.044,557.000,1514.000,1554.000,1580.000,2690.000
7,ID,9446516.000,47047120.551,15498072.866,1.000,46670724.000,51695384.000,54868808.000,61299420.000
8,InnovativeEmissionReductionsWltp,9446516.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


<class 'pandas.io.formats.style.Styler'>


,CommercialName,FuelType,CategoryOf,InnovativeTechnology,Co2EmissionsWltp,EnginePower,ElectricRange,InnovativeEmissionReductionsWltp,MassRunningOrder,BaseWheel,Pool,Count,Share
23153,CLIO,PETROL,M1,e9 28 29,118.0,67.0,0.0,0.0,1157.0,2583.0,RENAULT-NISSAN-MITSUBISHI,29437,6.532961e-03
17520,C3,PETROL,M1,e2 28 29,123.0,61.0,0.0,0.0,1055.0,2539.0,STELLANTIS,24891,5.524066e-03
1523,208,PETROL,M1,e2 28 29,118.0,74.0,0.0,0.0,1165.0,2540.0,STELLANTIS,20311,4.507626e-03
90921,TOYOTA YARIS,PETROL,M1,E6 28,92.0,68.0,0.0,0.0,1160.0,2560.0,MAZDA-SUBARU-SUZUKI-TOYOTA,17100,3.795007e-03
1193,2008,PETROL,M1,e2 28 29,136.0,96.0,0.0,0.0,1280.0,2605.0,STELLANTIS,16720,3.710674e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100306,Z4 SDRIVE30I,PETROL,M1,e24 28 29,163.0,190.0,0.0,0.0,1505.0,2470.0,BMW,1,2.219303e-07
100308,Z4 SDRIVE30I,PETROL,M1,e24 3 28,161.0,190.0,0.0,0.0,1505.0,2470.0,BMW,1,2.219303e-07
100313,Z4 SDRIVE30I G29 B48,PETROL,M1,e24 28,161.0,190.0,0.0,0.0,1505.0,2470.0,BMW,1,2.219303e-07
57,116D,DIESEL,M1,E24 29 37,125.0,85.0,0.0,0.0,1450.0,2670.0,BMW,1,2.219303e-07


In [3]:
co2.styles.display_describe(gb)

Save: True
Saved None styled DataFrame to output/tables


,Colonne,count,mean,std,min,25%,50%,75%,max
0,Co2EmissionsWltp,100365.000,154.620,35.252,0.000,131.000,145.000,169.000,506.000
1,EnginePower,100365.000,125.414,57.221,10.000,91.000,110.000,140.000,999.000
2,ElectricRange,100365.000,0.082,3.558,0.000,0.000,0.000,0.000,317.000
3,InnovativeEmissionReductionsWltp,100365.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,MassRunningOrder,100365.000,1625.257,322.918,935.000,1395.000,1574.000,1790.000,3275.000
5,BaseWheel,100365.000,2753.633,193.135,984.000,2636.000,2700.000,2836.000,4455.000
6,Count,100365.000,44.895,301.917,1.000,1.000,3.000,15.000,29437.000
7,Share,100365.000,0.000,0.000,0.000,0.000,0.000,0.000,0.007


<class 'pandas.io.formats.style.Styler'>


In [6]:
gb = co2.data.dummify_all_categoricals(gb, dummy_columns=['Pool', 'FuelType', 'CategoryOf'])

co2.data.save_processed_data(gb, classification=False, pickle=False)

Data saved to ../data/processed/regression/co2_data_regression_


In [5]:
gb.sort_values(by=['Count'], ascending=False)   

,CommercialName,InnovativeTechnology,Co2EmissionsWltp,EnginePower,ElectricRange,InnovativeEmissionsReductionWltp,MassRunningOrder,BaseWheel,Count,Share,...,FuelType_ETHANOL,FuelType_HYDROGEN,FuelType_LPG,FuelType_NATURALGAS,FuelType_NG,FuelType_PETROL,FuelType_PETROL/ELECTRIC,FuelType_UNKNOWN,CategoryOf_M1,CategoryOf_M1G
209805,SUPERB,0,36.0,115.0,51.0,0.0,1752.0,2836.0,4272,2.191366e-05,...,0,0,0,0,0,0,1,0,1,0
209836,SUPERB,0,39.0,115.0,50.0,0.0,1752.0,2836.0,4272,2.191366e-05,...,0,0,0,0,0,0,1,0,1,0
209835,SUPERB,0,38.0,115.0,63.0,0.0,1752.0,2836.0,4272,2.191366e-05,...,0,0,0,0,0,0,1,0,1,0
209834,SUPERB,0,38.0,115.0,52.0,0.0,1752.0,2836.0,4272,2.191366e-05,...,0,0,0,0,0,0,1,0,1,0
209833,SUPERB,0,38.0,115.0,51.0,0.0,1899.0,2836.0,4272,2.191366e-05,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205544,STONIC/GOLD/UVO/1.25 MPI/MT5/8,1,137.0,61.0,0.0,1.0,1155.0,2580.0,1,5.129603e-09,...,0,0,0,0,0,1,0,0,1,0
205550,STONIC/OE21/1.0 TGDI/MT6/100,1,125.0,73.0,0.0,1.4,1195.0,2580.0,1,5.129603e-09,...,0,0,0,0,0,1,0,0,1,0
205551,STONIC/SILBER LIMITED/1.0 TGDI,1,132.0,73.0,0.0,1.4,1195.0,2580.0,1,5.129603e-09,...,0,0,0,0,0,1,0,0,1,0
72796,D,0,237.0,183.0,0.0,0.0,2695.0,3022.0,1,5.129603e-09,...,0,0,0,0,0,0,0,0,0,1


In [11]:
gb[gb['CommercialName'] == 'CLIO'].sort_values(by='Count', ascending=False)

,CommercialName,InnovativeTechnology,Co2EmissionsWltp,EnginePower,ElectricRange,InnovativeEmissionsReductionWltp,MassRunningOrder,BaseWheel,Count,Share,...,FuelType_ETHANOL,FuelType_HYDROGEN,FuelType_LPG,FuelType_NATURALGAS,FuelType_NG,FuelType_PETROL,FuelType_PETROL/ELECTRIC,FuelType_UNKNOWN,CategoryOf_M1,CategoryOf_M1G
63054,CLIO,1,130.0,67.0,0.0,1.9,1201.0,2583.0,1092,0.000006,...,0,0,0,0,0,0,0,1,1,0
61963,CLIO,0,94.0,85.0,0.0,0.0,1264.0,2583.0,1092,0.000006,...,0,0,0,0,0,0,0,0,1,0
61964,CLIO,0,109.0,63.0,0.0,0.0,1264.0,2583.0,1092,0.000006,...,0,0,0,0,0,0,0,0,1,0
61965,CLIO,0,109.0,63.0,0.0,0.0,1284.0,2583.0,1092,0.000006,...,0,0,0,0,0,0,0,0,1,0
63015,CLIO,1,131.0,74.0,0.0,1.0,1187.0,2583.0,1092,0.000006,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61970,CLIO,0,110.0,63.0,0.0,0.0,1272.0,2583.0,1092,0.000006,...,0,0,0,0,0,0,0,0,1,0
61971,CLIO,0,110.0,63.0,0.0,0.0,1289.0,2583.0,1092,0.000006,...,0,0,0,0,0,0,0,0,1,0
61972,CLIO,0,110.0,63.0,0.0,0.0,1290.0,2583.0,1092,0.000006,...,0,0,0,0,0,0,0,0,1,0
61973,CLIO,0,110.0,63.0,0.0,0.0,1293.0,2583.0,1092,0.000006,...,0,0,0,0,0,0,0,0,1,0
